### 1. A

In [1]:
import threading
import time
import random
import logging

BUFFER_SIZE = 30
PRODUCER_SLEEP_MIN = 1.0
PRODUCER_SLEEP_MAX = 3.0
CONSUMER_SLEEP_MIN = 1.0
CONSUMER_SLEEP_MAX = 3.0
RUN_DURATION = 20
NUM_CONSUMERS = 5

LOG_FORMAT = '%(threadName)-17s %(levelname)-8s %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)

buffer = []
buffer_lock = threading.Lock()
empty_slots = threading.Semaphore(BUFFER_SIZE)
filled_slots = threading.Semaphore(0)

stop_threads = False

def producer(first, last):
    global stop_threads
    item_id = 0
    while not stop_threads:
        sleep_time = random.uniform(first, last)
        time.sleep(sleep_time)

        acquired = empty_slots.acquire(timeout=1)
        if not acquired:
            logging.info("Producer: Timeout waiting for empty slot.")
            continue

        if stop_threads:
            empty_slots.release()
            break

        item = f"item-{item_id}"
        with buffer_lock:
            buffer.append(item)
            logging.info(f"Produced: {item}")
            item_id += 1

        filled_slots.release()

    logging.info("Producer stopping...")

def consumer(consumer_id, first, last):
    global stop_threads
    while not stop_threads:
        sleep_time = random.uniform(first, last)
        time.sleep(sleep_time)

        acquired = filled_slots.acquire(timeout=1)
        if not acquired:
            if stop_threads:
                break
            logging.info(f"Consumer {consumer_id}: Timeout waiting for items.")
            continue

        if stop_threads:
            filled_slots.release()
            break

        with buffer_lock:
            if buffer:
                item = buffer.pop(0)
                logging.info(f"Consumer {consumer_id} consumed: {item}")
            else:
                logging.info(f"Consumer {consumer_id}: Buffer was empty after acquiring semaphore.")
                filled_slots.release()
                continue

        empty_slots.release()

    logging.info(f"Consumer {consumer_id} stopping...")

def timer_thread(duration):
    global stop_threads
    time.sleep(duration)
    stop_threads = True
    logging.info("Timer finished, requesting all threads to stop...")

def main():
    t_producer = threading.Thread(target=producer, args=(PRODUCER_SLEEP_MIN, PRODUCER_SLEEP_MAX), name="Producer")
    t_producer.start()

    consumers = []
    for i in range(NUM_CONSUMERS):
        t = threading.Thread(target=consumer, args=(i, CONSUMER_SLEEP_MIN, CONSUMER_SLEEP_MAX), name=f"Consumer-{i}")
        t.start()
        consumers.append(t)

    t_timer = threading.Thread(target=timer_thread, args=(RUN_DURATION,), name="Timer")
    t_timer.start()

    t_timer.join()
    t_producer.join()
    for t in consumers:
        t.join()

    logging.info("All threads terminated, program finished.")

if __name__ == "__main__":
    main()


Producer          INFO     Produced: item-0
Consumer-2        INFO     Consumer 2 consumed: item-0
Consumer-1        INFO     Consumer 1: Timeout waiting for items.
Consumer-3        INFO     Consumer 3: Timeout waiting for items.
Consumer-0        INFO     Consumer 0: Timeout waiting for items.
Consumer-4        INFO     Consumer 4: Timeout waiting for items.
Producer          INFO     Produced: item-1
Consumer-1        INFO     Consumer 1 consumed: item-1
Consumer-2        INFO     Consumer 2: Timeout waiting for items.
Producer          INFO     Produced: item-2
Consumer-3        INFO     Consumer 3 consumed: item-2
Consumer-0        INFO     Consumer 0: Timeout waiting for items.
Producer          INFO     Produced: item-3
Consumer-4        INFO     Consumer 4 consumed: item-3
Consumer-2        INFO     Consumer 2: Timeout waiting for items.
Consumer-3        INFO     Consumer 3: Timeout waiting for items.
Consumer-1        INFO     Consumer 1: Timeout waiting for items.
Consumer-4

# 1.B

In [2]:
BUFFER_SIZE = 30
PRODUCER_SLEEP_MIN = 0.1
PRODUCER_SLEEP_MAX = 0.3
CONSUMER_SLEEP_MIN = 3.0
CONSUMER_SLEEP_MAX = 4.0
RUN_DURATION = 20
NUM_CONSUMERS = 5

LOG_FORMAT = '%(threadName)-17s %(levelname)-8s %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)

buffer = []
buffer_lock = threading.Lock()
empty_slots = threading.Semaphore(BUFFER_SIZE)
filled_slots = threading.Semaphore(0)

stop_threads = False
def main():
    t_producer = threading.Thread(target=producer, args=(PRODUCER_SLEEP_MIN, PRODUCER_SLEEP_MAX), name="Producer")
    t_producer.start()

    consumers = []
    for i in range(NUM_CONSUMERS):
        t = threading.Thread(target=consumer, args=(i, CONSUMER_SLEEP_MIN, CONSUMER_SLEEP_MAX), name=f"Consumer-{i}")
        t.start()
        consumers.append(t)

    t_timer = threading.Thread(target=timer_thread, args=(RUN_DURATION,), name="Timer")
    t_timer.start()

    t_timer.join()
    t_producer.join()
    for t in consumers:
        t.join()

    logging.info("All threads terminated, program finished.")

if __name__ == "__main__":
    main()

Producer          INFO     Produced: item-0
Producer          INFO     Produced: item-1
Producer          INFO     Produced: item-2
Producer          INFO     Produced: item-3
Producer          INFO     Produced: item-4
Producer          INFO     Produced: item-5
Producer          INFO     Produced: item-6
Producer          INFO     Produced: item-7
Producer          INFO     Produced: item-8
Producer          INFO     Produced: item-9
Producer          INFO     Produced: item-10
Producer          INFO     Produced: item-11
Producer          INFO     Produced: item-12
Producer          INFO     Produced: item-13
Producer          INFO     Produced: item-14
Consumer-4        INFO     Consumer 4 consumed: item-0
Producer          INFO     Produced: item-15
Consumer-2        INFO     Consumer 2 consumed: item-1
Producer          INFO     Produced: item-16
Consumer-3        INFO     Consumer 3 consumed: item-2
Consumer-0        INFO     Consumer 0 consumed: item-3
Producer          INFO   

# 1.C

In [3]:
BUFFER_SIZE = 30
PRODUCER_SLEEP_MIN = 0.2
PRODUCER_SLEEP_MAX = 0.5
CONSUMER_SLEEP_MIN = 1.5
CONSUMER_SLEEP_MAX = 3.0
RUN_DURATION = 20
NUM_CONSUMERS = 5

LOG_FORMAT = '%(threadName)-17s %(levelname)-8s %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)

buffer = []
buffer_lock = threading.Lock()
empty_slots = threading.Semaphore(BUFFER_SIZE)
filled_slots = threading.Semaphore(0)

stop_threads = False
def main():
    t_producer = threading.Thread(target=producer, args=(PRODUCER_SLEEP_MIN, PRODUCER_SLEEP_MAX), name="Producer")
    t_producer.start()

    consumers = []
    for i in range(NUM_CONSUMERS):
        t = threading.Thread(target=consumer, args=(i, CONSUMER_SLEEP_MIN, CONSUMER_SLEEP_MAX), name=f"Consumer-{i}")
        t.start()
        consumers.append(t)

    t_timer = threading.Thread(target=timer_thread, args=(RUN_DURATION,), name="Timer")
    t_timer.start()

    t_timer.join()
    t_producer.join()
    for t in consumers:
        t.join()

    logging.info("All threads terminated, program finished.")

if __name__ == "__main__":
    main()

Producer          INFO     Produced: item-0
Producer          INFO     Produced: item-1
Producer          INFO     Produced: item-2
Producer          INFO     Produced: item-3
Consumer-0        INFO     Consumer 0 consumed: item-0
Consumer-1        INFO     Consumer 1 consumed: item-1
Consumer-3        INFO     Consumer 3 consumed: item-2
Producer          INFO     Produced: item-4
Consumer-4        INFO     Consumer 4 consumed: item-3
Consumer-2        INFO     Consumer 2 consumed: item-4
Producer          INFO     Produced: item-5
Producer          INFO     Produced: item-6
Consumer-1        INFO     Consumer 1 consumed: item-5
Producer          INFO     Produced: item-7
Producer          INFO     Produced: item-8
Consumer-3        INFO     Consumer 3 consumed: item-6
Consumer-0        INFO     Consumer 0 consumed: item-7
Producer          INFO     Produced: item-9
Consumer-4        INFO     Consumer 4 consumed: item-8
Producer          INFO     Produced: item-10
Consumer-2        IN

# 1.A

In [11]:
import threading
import queue
import time
import random

BUFFER_SIZE = 30
q = queue.Queue(BUFFER_SIZE)
stop_threads = False
def producer(first, last):
    item_id = 0
    while True:
        if stop_threads:
            print("Producer stopping...")
            break
        
        time.sleep(random.uniform(first, last))
        item = f"item-{item_id}"
        if not q.full():
            q.put(item, timeout=1)
            print(f"Produced: {item}")
            item_id += 1
        else:
            print(f"producer: queue is full, waiting for 1 second")
            time.sleep(1)


def consumer(consumer_id, first, last):
    while True:
        if stop_threads:
            print(f"Consumer {consumer_id} stopping...")
            break
        
        time.sleep(random.uniform(first, last))
        if not q.empty():
                
            item = q.get(timeout=1)
            print(f"Consumer {consumer_id} consumed: {item}")
            q.task_done()
        else:

            print(f"Consumer {consumer_id} queue is empty, waiting for 1 second")
            time.sleep(1)


def timer_thread(duration):
    time.sleep(duration)
    global stop_threads
    stop_threads = True
    print("Timer finished, requesting all threads to stop...")


if __name__ == "__main__":
    t_producer = threading.Thread(target=producer, args=(1., 3.,))
    t_producer.start()

    consumers = []
    for i in range(5):
        t = threading.Thread(target=consumer, args=(i, 1.0, 3.0,))
        t.start()
        consumers.append(t)

    t_timer = threading.Thread(target=timer_thread, args=(20,))
    t_timer.start()

    t_producer.join()
    for t in consumers:
        t.join()
    t_timer.join()

    print("All threads terminated, program finished.")

Consumer 3 queue is empty, waiting for 1 second
Consumer 0 queue is empty, waiting for 1 second
Consumer 4 queue is empty, waiting for 1 second
Produced: item-0
Consumer 2 consumed: item-0
Consumer 1 queue is empty, waiting for 1 second
Produced: item-1
Consumer 3 consumed: item-1
Consumer 2 queue is empty, waiting for 1 second
Consumer 0 queue is empty, waiting for 1 second
Consumer 3 queue is empty, waiting for 1 second
Consumer 4 queue is empty, waiting for 1 second
Consumer 1 queue is empty, waiting for 1 second
Produced: item-2
Consumer 2 consumed: item-2
Consumer 3 queue is empty, waiting for 1 second
Consumer 0 queue is empty, waiting for 1 second
Consumer 1 queue is empty, waiting for 1 second
Consumer 4 queue is empty, waiting for 1 second
Produced: item-3
Consumer 2 consumed: item-3
Consumer 0 queue is empty, waiting for 1 second
Consumer 3 queue is empty, waiting for 1 second
Produced: item-4
Consumer 1 consumed: item-4
Consumer 2 queue is empty, waiting for 1 second
Consume

### 1. B

In [ ]:
stop_threads = False
q = queue.Queue(BUFFER_SIZE)
if __name__ == "__main__":
    t_producer = threading.Thread(target=producer, args=(0.1, 0.3,))
    t_producer.start()

    consumers = []
    for i in range(5):
        t = threading.Thread(target=consumer, args=(i, 3.0, 4.0,))
        t.start()
        consumers.append(t)

    t_timer = threading.Thread(target=timer_thread, args=(20,))
    t_timer.start()

    t_producer.join()
    for t in consumers:
        t.join()
    t_timer.join()

    print("All threads terminated, program finished.")


Produced: item-0
Produced: item-1
Produced: item-2
Produced: item-3
Produced: item-4
Produced: item-5
Produced: item-6
Produced: item-7
Produced: item-8
Produced: item-9
Produced: item-10
Produced: item-11
Produced: item-12
Produced: item-13
Produced: item-14
Produced: item-15
Consumer 3 consumed: item-0
Produced: item-16
Consumer 0 consumed: item-1
Consumer 4 consumed: item-2
Produced: item-17
Produced: item-18
Consumer 2 consumed: item-3
Consumer 1 consumed: item-4
Produced: item-19
Produced: item-20
Produced: item-21
Produced: item-22
Produced: item-23
Produced: item-24
Produced: item-25
Produced: item-26
Produced: item-27
Produced: item-28
Produced: item-29
Produced: item-30
Consumer 4 consumed: item-5
Produced: item-31
Produced: item-32
Consumer 2 consumed: item-6
Consumer 3 consumed: item-7
Consumer 0 consumed: item-8
Consumer 1 consumed: item-9
Produced: item-33
Produced: item-34
Produced: item-35
Produced: item-36
Produced: item-37
Produced: item-38
Produced: item-39
producer: 

### 1. C

In [15]:
stop_threads = False
q = queue.Queue(BUFFER_SIZE)

if __name__ == "__main__":
    t_producer = threading.Thread(target=producer, args=(.2, .5,))
    t_producer.start()

    consumers = []
    for i in range(5):
        t = threading.Thread(target=consumer, args=(i, 1.5, 3.0,))
        t.start()
        consumers.append(t)

    t_timer = threading.Thread(target=timer_thread, args=(20,))
    t_timer.start()

    t_producer.join()
    for t in consumers:
        t.join()
    t_timer.join()

    print("All threads terminated, program finished.")

Produced: item-0
Produced: item-1
Produced: item-2
Produced: item-3
Consumer 3 consumed: item-0
Consumer 1 consumed: item-1Produced: item-4

Produced: item-5
Produced: item-6
Consumer 2 consumed: item-2
Consumer 0 consumed: item-3
Consumer 4 consumed: item-4
Produced: item-7
Consumer 3 consumed: item-5
Produced: item-8
Produced: item-9
Produced: item-10
Consumer 1 consumed: item-6
Produced: item-11
Produced: item-12
Produced: item-13
Produced: item-14
Consumer 2 consumed: item-7
Consumer 4 consumed: item-8
Produced: item-15
Consumer 3 consumed: item-9
Consumer 0 consumed: item-10
Produced: item-16
Produced: item-17
Produced: item-18
Consumer 1 consumed: item-11
Produced: item-19
Consumer 2 consumed: item-12
Produced: item-20
Produced: item-21Consumer 3 consumed: item-13

Produced: item-22
Consumer 4 consumed: item-14
Produced: item-23
Produced: item-24
Consumer 0 consumed: item-15
Consumer 2 consumed: item-16
Produced: item-25
Produced: item-26
Consumer 1 consumed: item-17
Produced: it

### 2. A

In [4]:
import logging
import threading
import time
import random

LOG_FORMAT = '%(threadName)-17s %(levelname)-8s %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)

BUFFER_SIZE = 30
PRODUCER_SLEEP_MIN = 1.0
PRODUCER_SLEEP_MAX = 2.0
CONSUMER_SLEEP_MIN = 0.1
CONSUMER_SLEEP_MAX = 0.5
RUN_DURATION = 20
NUM_CONSUMERS = 5
NUM_PRODUCERS = 3

buffer = []
buffer_lock = threading.Lock()
empty_slots = threading.Semaphore(BUFFER_SIZE)
filled_slots = threading.Semaphore(0)

stop_threads = False

def producer(first, last):
    global stop_threads
    item_id = 0
    while not stop_threads:
        sleep_time = random.uniform(first, last)
        time.sleep(sleep_time)
        acquired = empty_slots.acquire(timeout=1)
        if not acquired:
            logging.info("Producer: Timeout waiting for empty slot.")
            continue
        if stop_threads:
            empty_slots.release()
            break
        item = f"item-{item_id}"
        with buffer_lock:
            buffer.append(item)
            logging.info(f"Produced: {item}")
            item_id += 1
        filled_slots.release()
    logging.info("Producer stopping...")

def consumer(consumer_id, first, last):
    global stop_threads
    while not stop_threads:
        sleep_time = random.uniform(first, last)
        time.sleep(sleep_time)
        acquired = filled_slots.acquire(timeout=1)
        if not acquired:
            if stop_threads:
                break
            logging.info(f"Consumer {consumer_id}: Timeout waiting for items.")
            continue
        if stop_threads:
            filled_slots.release()
            break
        with buffer_lock:
            if buffer:
                item = buffer.pop(0)
                logging.info(f"Consumer {consumer_id} consumed: {item}")
            else:
                logging.info(f"Consumer {consumer_id}: Buffer was empty after acquiring semaphore.")
                filled_slots.release()
                continue
        empty_slots.release()
    logging.info(f"Consumer {consumer_id} stopping...")

def timer_thread(duration):
    global stop_threads
    time.sleep(duration)
    stop_threads = True
    logging.info("Timer finished, requesting all threads to stop...")

def main():
    producers = []
    for i in range(NUM_PRODUCERS):
        t = threading.Thread(target=producer, args=(PRODUCER_SLEEP_MIN, PRODUCER_SLEEP_MAX), name=f"Producer-{i}")
        t.start()
        producers.append(t)
    consumers = []
    for i in range(NUM_CONSUMERS):
        t = threading.Thread(target=consumer, args=(i, CONSUMER_SLEEP_MIN, CONSUMER_SLEEP_MAX), name=f"Consumer-{i}")
        t.start()
        consumers.append(t)
    t_timer = threading.Thread(target=timer_thread, args=(RUN_DURATION,), name="Timer")
    t_timer.start()
    t_timer.join()
    for t in producers:
        t.join()
    for t in consumers:
        t.join()
    logging.info("All threads terminated, program finished.")

if __name__ == "__main__":
    main()


Consumer-1        INFO     Consumer 1: Timeout waiting for items.
Producer-0        INFO     Produced: item-0
Consumer-3        INFO     Consumer 3 consumed: item-0
Consumer-2        INFO     Consumer 2: Timeout waiting for items.
Consumer-0        INFO     Consumer 0: Timeout waiting for items.
Consumer-4        INFO     Consumer 4: Timeout waiting for items.
Producer-2        INFO     Produced: item-0
Consumer-3        INFO     Consumer 3 consumed: item-0
Producer-1        INFO     Produced: item-0
Consumer-1        INFO     Consumer 1 consumed: item-0
Consumer-0        INFO     Consumer 0: Timeout waiting for items.
Consumer-2        INFO     Consumer 2: Timeout waiting for items.
Consumer-4        INFO     Consumer 4: Timeout waiting for items.
Consumer-3        INFO     Consumer 3: Timeout waiting for items.
Producer-0        INFO     Produced: item-1
Consumer-1        INFO     Consumer 1 consumed: item-1
Producer-1        INFO     Produced: item-1
Consumer-2        INFO     Consu

# 2.B

In [5]:
BUFFER_SIZE = 30
PRODUCER_SLEEP_MIN = 0.1
PRODUCER_SLEEP_MAX = 0.3
CONSUMER_SLEEP_MIN = 3.0
CONSUMER_SLEEP_MAX = 4.0
RUN_DURATION = 20
NUM_CONSUMERS = 5
NUM_PRODUCERS = 3

buffer = []
buffer_lock = threading.Lock()
empty_slots = threading.Semaphore(BUFFER_SIZE)
filled_slots = threading.Semaphore(0)

stop_threads = False

def main():
    producers = []
    for i in range(NUM_PRODUCERS):
        t = threading.Thread(target=producer, args=(PRODUCER_SLEEP_MIN, PRODUCER_SLEEP_MAX), name=f"Producer-{i}")
        t.start()
        producers.append(t)
    consumers = []
    for i in range(NUM_CONSUMERS):
        t = threading.Thread(target=consumer, args=(i, CONSUMER_SLEEP_MIN, CONSUMER_SLEEP_MAX), name=f"Consumer-{i}")
        t.start()
        consumers.append(t)
    t_timer = threading.Thread(target=timer_thread, args=(RUN_DURATION,), name="Timer")
    t_timer.start()
    t_timer.join()
    for t in producers:
        t.join()
    for t in consumers:
        t.join()
    logging.info("All threads terminated, program finished.")

if __name__ == "__main__":
    main()

Producer-2        INFO     Produced: item-0
Producer-0        INFO     Produced: item-0
Producer-1        INFO     Produced: item-0
Producer-0        INFO     Produced: item-1
Producer-1        INFO     Produced: item-1
Producer-0        INFO     Produced: item-2
Producer-2        INFO     Produced: item-1
Producer-1        INFO     Produced: item-2
Producer-0        INFO     Produced: item-3
Producer-2        INFO     Produced: item-2
Producer-0        INFO     Produced: item-4
Producer-2        INFO     Produced: item-3
Producer-1        INFO     Produced: item-3
Producer-0        INFO     Produced: item-5
Producer-2        INFO     Produced: item-4
Producer-1        INFO     Produced: item-4
Producer-1        INFO     Produced: item-5
Producer-2        INFO     Produced: item-5
Producer-0        INFO     Produced: item-6
Producer-2        INFO     Produced: item-6
Producer-2        INFO     Produced: item-7
Producer-1        INFO     Produced: item-6
Producer-0        INFO     Produ

# 2.C

In [6]:
BUFFER_SIZE = 30
PRODUCER_SLEEP_MIN = 0.2
PRODUCER_SLEEP_MAX = 0.4
CONSUMER_SLEEP_MIN = 0.45
CONSUMER_SLEEP_MAX = 0.6
RUN_DURATION = 20
NUM_CONSUMERS = 5
NUM_PRODUCERS = 3

buffer = []
buffer_lock = threading.Lock()
empty_slots = threading.Semaphore(BUFFER_SIZE)
filled_slots = threading.Semaphore(0)

stop_threads = False

def main():
    producers = []
    for i in range(NUM_PRODUCERS):
        t = threading.Thread(target=producer, args=(PRODUCER_SLEEP_MIN, PRODUCER_SLEEP_MAX), name=f"Producer-{i}")
        t.start()
        producers.append(t)
    consumers = []
    for i in range(NUM_CONSUMERS):
        t = threading.Thread(target=consumer, args=(i, CONSUMER_SLEEP_MIN, CONSUMER_SLEEP_MAX), name=f"Consumer-{i}")
        t.start()
        consumers.append(t)
    t_timer = threading.Thread(target=timer_thread, args=(RUN_DURATION,), name="Timer")
    t_timer.start()
    t_timer.join()
    for t in producers:
        t.join()
    for t in consumers:
        t.join()
    logging.info("All threads terminated, program finished.")

if __name__ == "__main__":
    main()

Producer-1        INFO     Produced: item-0
Producer-2        INFO     Produced: item-0
Producer-0        INFO     Produced: item-0
Consumer-4        INFO     Consumer 4 consumed: item-0
Consumer-0        INFO     Consumer 0 consumed: item-0
Producer-1        INFO     Produced: item-1
Consumer-1        INFO     Consumer 1 consumed: item-0
Consumer-3        INFO     Consumer 3 consumed: item-1
Producer-2        INFO     Produced: item-1
Consumer-2        INFO     Consumer 2 consumed: item-1
Producer-0        INFO     Produced: item-1
Producer-1        INFO     Produced: item-2
Producer-2        INFO     Produced: item-2
Consumer-4        INFO     Consumer 4 consumed: item-1
Consumer-0        INFO     Consumer 0 consumed: item-2
Consumer-3        INFO     Consumer 3 consumed: item-2
Producer-0        INFO     Produced: item-2
Consumer-1        INFO     Consumer 1 consumed: item-2
Producer-1        INFO     Produced: item-3
Producer-2        INFO     Produced: item-3
Consumer-2        INF

# 2.A

In [2]:
import threading
import queue
import time
import random

BUFFER_SIZE = 30
q = queue.Queue(BUFFER_SIZE)
stop_threads = False

def producer(prod_id, first, last):
    item_id = 0
    while True:
        if stop_threads:
            print(f"Producer {prod_id} stopping...")
            break
        
        time.sleep(random.uniform(first, last))
        item = f"P{prod_id}-item-{item_id}"
        if not q.full():
            q.put(item, timeout=1)
            print(f"Producer {prod_id} produced: {item}")
            item_id += 1
        else:
            print(f"Producer {prod_id}: queue is full, waiting for 1 second")
            time.sleep(1)

def consumer(consumer_id, first, last):
    while True:
        if stop_threads:
            print(f"Consumer {consumer_id} stopping...")
            break
        
        time.sleep(random.uniform(first, last))
        if not q.empty():
            item = q.get(timeout=1)
            print(f"Consumer {consumer_id} consumed: {item}")
            q.task_done()
        else:
            print(f"Consumer {consumer_id}: queue is empty, waiting for 1 second")
            time.sleep(1)

def timer_thread(duration):
    time.sleep(duration)
    global stop_threads
    stop_threads = True
    print("Timer finished, requesting all threads to stop...")

if __name__ == "__main__":
    producers = []
    for i in range(3):
        t = threading.Thread(target=producer, args=(i, 1., 2.,))
        t.start()
        producers.append(t)
    
    consumers = []
    for i in range(5):
        t = threading.Thread(target=consumer, args=(i, 0.1, 0.5,))
        t.start()
        consumers.append(t)

    t_timer = threading.Thread(target=timer_thread, args=(20,))
    t_timer.start()

    for t in producers:
        t.join()
    for t in consumers:
        t.join()
    t_timer.join()

    print("All threads terminated, program finished.")

Consumer 1: queue is empty, waiting for 1 second
Consumer 0: queue is empty, waiting for 1 second
Consumer 3: queue is empty, waiting for 1 second
Consumer 4: queue is empty, waiting for 1 second
Consumer 2: queue is empty, waiting for 1 second
Producer 2 produced: P2-item-0
Consumer 1 consumed: P2-item-0
Producer 1 produced: P1-item-0
Consumer 4 consumed: P1-item-0
Consumer 3: queue is empty, waiting for 1 second
Consumer 2: queue is empty, waiting for 1 second
Consumer 0: queue is empty, waiting for 1 second
Consumer 1: queue is empty, waiting for 1 second
Producer 0 produced: P0-item-0
Consumer 4 consumed: P0-item-0
Consumer 4: queue is empty, waiting for 1 second
Producer 2 produced: P2-item-1
Producer 0 produced: P0-item-1
Consumer 2 consumed: P2-item-1
Consumer 3 consumed: P0-item-1
Consumer 0: queue is empty, waiting for 1 second
Producer 1 produced: P1-item-1
Consumer 2 consumed: P1-item-1
Consumer 1: queue is empty, waiting for 1 second
Consumer 3: queue is empty, waiting for 

### 2. B

In [3]:
q = queue.Queue(BUFFER_SIZE)
stop_threads = False
if __name__ == "__main__":
    producers = []
    for i in range(3):
        t = threading.Thread(target=producer, args=(i, 0.1, 0.3,))
        t.start()
        producers.append(t)
    
    consumers = []
    for i in range(5):
        t = threading.Thread(target=consumer, args=(i, 3.0, 4.0,))
        t.start()
        consumers.append(t)

    t_timer = threading.Thread(target=timer_thread, args=(20,))
    t_timer.start()

    for t in producers:
        t.join()
    for t in consumers:
        t.join()
    t_timer.join()

    print("All threads terminated, program finished.")


Producer 0 produced: P0-item-0
Producer 2 produced: P2-item-0
Producer 1 produced: P1-item-0
Producer 2 produced: P2-item-1
Producer 0 produced: P0-item-1
Producer 1 produced: P1-item-1
Producer 2 produced: P2-item-2
Producer 0 produced: P0-item-2
Producer 1 produced: P1-item-2
Producer 2 produced: P2-item-3
Producer 0 produced: P0-item-3
Producer 1 produced: P1-item-3
Producer 0 produced: P0-item-4
Producer 2 produced: P2-item-4
Producer 1 produced: P1-item-4
Producer 0 produced: P0-item-5
Producer 2 produced: P2-item-5
Producer 0 produced: P0-item-6
Producer 1 produced: P1-item-5
Producer 2 produced: P2-item-6
Producer 1 produced: P1-item-6
Producer 0 produced: P0-item-7
Producer 2 produced: P2-item-7
Producer 1 produced: P1-item-7
Producer 1 produced: P1-item-8Producer 0 produced: P0-item-8

Producer 2 produced: P2-item-8
Producer 2 produced: P2-item-9
Producer 0 produced: P0-item-9
Producer 1 produced: P1-item-9
Producer 1: queue is full, waiting for 1 second
Producer 2: queue is f

In [4]:
q = queue.Queue(BUFFER_SIZE)
stop_threads = False
if __name__ == "__main__":
    producers = []
    for i in range(3):
        t = threading.Thread(target=producer, args=(i, .2, .4,))
        t.start()
        producers.append(t)
    
    consumers = []
    for i in range(5):
        t = threading.Thread(target=consumer, args=(i, 0.45, 0.6,))
        t.start()
        consumers.append(t)

    t_timer = threading.Thread(target=timer_thread, args=(20,))
    t_timer.start()

    for t in producers:
        t.join()
    for t in consumers:
        t.join()
    t_timer.join()

    print("All threads terminated, program finished.")

Producer 0 produced: P0-item-0
Producer 1 produced: P1-item-0
Producer 2 produced: P2-item-0
Consumer 4 consumed: P0-item-0
Consumer 0 consumed: P1-item-0
Consumer 2 consumed: P2-item-0
Producer 0 produced: P0-item-1
Consumer 1 consumed: P0-item-1
Producer 2 produced: P2-item-1
Producer 1 produced: P1-item-1
Consumer 3 consumed: P2-item-1
Producer 1 produced: P1-item-2
Producer 2 produced: P2-item-2
Producer 0 produced: P0-item-2
Consumer 2 consumed: P1-item-1
Consumer 4 consumed: P1-item-2
Consumer 0 consumed: P2-item-2
Consumer 1 consumed: P0-item-2
Consumer 3: queue is empty, waiting for 1 second
Producer 1 produced: P1-item-3
Producer 0 produced: P0-item-3
Producer 2 produced: P2-item-3
Producer 1 produced: P1-item-4
Consumer 4 consumed: P1-item-3
Producer 0 produced: P0-item-4
Consumer 2 consumed: P0-item-3
Consumer 0 consumed: P2-item-3
Consumer 1 consumed: P1-item-4
Producer 2 produced: P2-item-4
Producer 1 produced: P1-item-5
Producer 0 produced: P0-item-5
Producer 1 produced: 

### 3

In [1]:
import threading
import time

sem1 = threading.Semaphore(1)
sem2 = threading.Semaphore(1)

def thread1():
    sem1.acquire()
    print("Thread 1 acquired sem1")
    time.sleep(1) 
    print("Thread 1 waiting for sem2...")
    sem2.acquire()
    print("Thread 1 acquired sem2")

def thread2():
    sem2.acquire()
    print("Thread 2 acquired sem2")
    time.sleep(1)
    print("Thread 2 waiting for sem1...")
    sem1.acquire()
    print("Thread 2 acquired sem1")

if __name__ == "__main__":
    t1 = threading.Thread(target=thread1)
    t2 = threading.Thread(target=thread2)

    t1.start()
    t2.start()

    time.sleep(5)
    print("Deadlock occurred.")


Thread 1 acquired sem1
Thread 2 acquired sem2
Thread 2 waiting for sem1...
Thread 1 waiting for sem2...
Deadlock occurred.


### 4

In [5]:
import threading
import time

sem = threading.Semaphore(0)

def P1():
    print('2 seconds sleep to ensure that P2 is not doing anything before P1')
    time.sleep(2)
    print("P1 is running first...")
    time.sleep(2)
    print("P1 finished, releasing semaphore for P2.")
    sem.release()

def P2():
    print("P2 waiting for P1 to finish...")
    sem.acquire()
    print("P2 is now running after P1.")

if __name__ == "__main__":
    t1 = threading.Thread(target=P1)
    t2 = threading.Thread(target=P2)

    t2.start()
    t1.start()

    t1.join()
    t2.join()

    print("P1 finished before P2, as required.")


P2 waiting for P1 to finish...
2 seconds sleep to ensure that P2 is not doing anything before P1
P1 is running first...
P1 finished, releasing semaphore for P2.
P2 is now running after P1.
P1 finished before P2, as required.
